In [3]:
from src.training.ModelTrainer import ModelTrainer
from src.datasets.shape_net.shape_net_v3_sdf import ShapeNetV3SDF
from src.utils.util import seed_all
from src.utils.visualizations import visualize_sdf_as_mesh
from tqdm.notebook import tqdm as tqdm
import numpy as np
import torch
%load_ext autoreload
%load_ext tensorboard
%autoreload 2
seed_all(111)
mem = torch.cuda.mem_get_info()
print("Memory Before: %s", mem)
trainer = ModelTrainer(dataset_type=ShapeNetV3SDF,
                       options={"tdm_notebook": True})

dataset = trainer.data_loader_handler.dataset
print("Dataset length: ", len(dataset))
mem = torch.cuda.mem_get_info()
print("Memory After: %s", mem)
model = trainer.model


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Memory Before: %s (1780154368, 10385358848)
Using device: cuda:0
{'name': 'globalPVQVAEVGG', 'description': 'plz', 'experiment_id': 'Vis', 'extra_notes': 'none', 'logs_dir': 'logs', 'is_train': True, 'device': 'cuda:0', 'batch_size': 16, 'num_workers': 16, 'test_size': 0.1, 'n_epochs': 50000, 'append_loss_every': 250, 'print_every': 500, 'validate_every': 1973, 'save_every': 1973, 'save_every_nepochs': 2, 'start_epoch': 0, 'start_iteration': 0, 'visualize_every': 1000, 'apply_metrics_every': 1973, 'load_ckpt': True, 'ckpt_path': 'logs/globalPVQVAEVGG/fullTrain/2024_07_03_12_16_57/checkpoints/epoch-latest.ckpt', 'use_scheduler': True, 'apply_metrics_batch_count': 5}
[*] Enc has Attn at i_level, i_block: 3, 0
Decoding of shape (1, 256, 8, 8, 8) = 131072 dimensions.
[*] Dec has Attn at i_level, i_block: 3, 0
Using

- Creating new directory logs/globalPVQVAEVGG/Vis/2024_07_04_10_40_22
- Creating new directory logs/globalPVQVAEVGG/Vis/2024_07_04_10_40_22/checkpoints
- Creating new directory logs/globalPVQVAEVGG/Vis/2024_07_04_10_40_22/tb
- Creating new directory logs/globalPVQVAEVGG/Vis/2024_07_04_10_40_22/visuals
- Creating new directory logs/globalPVQVAEVGG/Vis/2024_07_04_10_40_22/modelsummary


Model loaded from logs/globalPVQVAEVGG/fullTrain/2024_07_03_12_16_57/checkpoints/epoch-latest.ckpt
Model size: 110.092MB
Total_params: 28.9M
Dataset length:  6579
Memory After: %s (1780154368, 10385358848)


In [5]:
from src.training.SDFVisualizer import SDFVisualizer
from src.metrics.pytorch_3d_chamfer_dist import Pytorch3DChamferDistance
from src.metrics.signed_iou import SignedIou
from src.metrics.pytorch_3d_chamfer_dist import Pytorch3DChamferDistance
from src.metrics.signed_iou import SignedIou
chm = Pytorch3DChamferDistance()
iou = SignedIou()
path = "./src/testVisuals/vggPvqvaeGlobal"
data_loader = trainer.data_loader_handler.validation_dataloader
with torch.no_grad():
    chamferDistTotal = 0
    iouTotal = 0
    i = 0
    for index, batch in tqdm(enumerate(data_loader), total=len(data_loader)):
        ShapeNetV3SDF.move_batch_to_device(batch, "cuda:0")
        sdfs = batch['sdf']
        pred = model.inference(sdfs)
        visualizer = SDFVisualizer("cuda:0",path,"normal", 0, index)
        visualizer.visualize(pred)
        chamferDistTotal += chm.calc_batch(pred,sdfs)
        iouTotal += iou.calc_batch(pred, sdfs)
        i+=1
chamferDistTotal =  chamferDistTotal/(i)
iouTotal = iouTotal/(i)

  0%|          | 0/41 [00:00<?, ?it/s]

./src/testVisuals/vggPvqvaeGlobal/normal/epoch_0_iter_0.png saved
./src/testVisuals/vggPvqvaeGlobal/normal/epoch_0_iter_1.png saved
./src/testVisuals/vggPvqvaeGlobal/normal/epoch_0_iter_2.png saved
./src/testVisuals/vggPvqvaeGlobal/normal/epoch_0_iter_3.png saved
./src/testVisuals/vggPvqvaeGlobal/normal/epoch_0_iter_4.png saved
./src/testVisuals/vggPvqvaeGlobal/normal/epoch_0_iter_5.png saved
./src/testVisuals/vggPvqvaeGlobal/normal/epoch_0_iter_6.png saved
./src/testVisuals/vggPvqvaeGlobal/normal/epoch_0_iter_7.png saved
./src/testVisuals/vggPvqvaeGlobal/normal/epoch_0_iter_8.png saved
./src/testVisuals/vggPvqvaeGlobal/normal/epoch_0_iter_9.png saved
./src/testVisuals/vggPvqvaeGlobal/normal/epoch_0_iter_10.png saved
./src/testVisuals/vggPvqvaeGlobal/normal/epoch_0_iter_11.png saved
./src/testVisuals/vggPvqvaeGlobal/normal/epoch_0_iter_12.png saved
./src/testVisuals/vggPvqvaeGlobal/normal/epoch_0_iter_13.png saved
./src/testVisuals/vggPvqvaeGlobal/normal/epoch_0_iter_14.png saved
./src

In [6]:
iouTotal

tensor(0.7846, device='cuda:0')

In [ ]:
chamferDistTotal